# Соколов Никита ИУ6-54Б Вариант 2

In [1]:
import ast
import csv
import math
import os
import sys

from pyspark import SparkConf, SparkContext

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"
os.environ['_JAVA_OPTIONS'] = '-Dhadoop.security.authentication=simple -Duser.name=spark'

try:
    sc.stop()
except:
    pass

spark_config = SparkConf().setAppName("BigData_HW1").setMaster("local[*]")
sc = SparkContext(conf=spark_config)

Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Picked up _JAVA_OPTIONS: -Dhadoop.security.authentication=simple -Duser.name=spark
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/06 16:04:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
def compute_haversine(lat_start, lon_start, lat_end, lon_end):
    EARTH_RADIUS = 6371.0

    # преобразование в радианы
    phi_1 = math.radians(lat_start)
    lambda_1 = math.radians(lon_start)
    phi_2 = math.radians(lat_end)
    lambda_2 = math.radians(lon_end)

    delta_phi = phi_2 - phi_1
    delta_lambda = lambda_2 - lambda_1

    part_a = math.sin(delta_phi / 2)**2 + \
             math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2)**2
    
    part_c = 2 * math.atan2(math.sqrt(part_a), math.sqrt(1 - part_a))

    return EARTH_RADIUS * part_c

In [ ]:
def load_csv_data(source_path):
    parsed_items = []
    try:
        with open(source_path, 'r', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)

            for line in csv_reader:
                # проверка количества столбцов
                if len(line) > 13:
                    try:
                        p_name = line[1]
                        p_address = line[8]
                        p_lat = float(line[13])
                        p_lon = float(line[12])
                        parsed_items.append((p_name, p_address, p_lat, p_lon))
                    except ValueError:
                        pass
        return sc.parallelize(parsed_items)
    except Exception as error:
        print(f"Error reading file: {error}")
        return sc.emptyRDD()


places_rdd = load_csv_data('./places.csv')
places_rdd.take(5)

[('Шоколадница',
  'город Москва, Бауманская улица, дом 33/2, строение 1',
  55.77302899555391,
  37.678681276899106),
 ('МУ-МУ',
  'город Москва, Бауманская улица, дом 35/1',
  55.771586681103294,
  37.678960226636995),
 ('КОМБИНАТ ПИТАНИЯ МГТУ ИМ.Н.Э.БАУМАНА',
  'город Москва, Бригадирский переулок, дом 3-5',
  55.76545455297085,
  37.679609224948955),
 ('Дом 12',
  'город Москва, улица Казакова, дом 8А',
  55.762608795785084,
  37.661498425199866),
 ('Чито-Ра',
  'город Москва, улица Казакова, дом 10/2, строение 1',
  55.7622012817049,
  37.662845514876196)]

## Задание 1
- Рассчитайте средний рейтинг товаров из набора данных.

In [ ]:
def extract_metadata(raw_text):
    try:
        text_content = raw_text.strip()
        entry = ast.literal_eval(text_content)
        
        prod_asin = entry.get('asin', '0')
        prod_name = entry.get('title', 'unk')
        
        return (prod_asin, prod_name)
    except Exception:
        return None

metadata_rdd = sc.textFile('/home/beekfin/sna23mk048/BigData/meta_Electronics.json') \
                 .map(extract_metadata) \
                 .filter(lambda x: x is not None)

metadata_rdd.take(5)

[('0132793040',
  'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'),
 ('0321732944',
  'Kelby Training DVD: Adobe Photoshop CS5 Crash Course By Matt Kloskowski'),
 ('0439886341', 'Digital Organizer and Messenger'),
 ('0511189877', 'CLIKR-5 Time Warner Cable Remote Control UR5U-8780L'),
 ('0528881469',
  'Rand McNally 528881469 7-inch Intelliroute TND 700 Truck GPS')]

In [ ]:
def extract_reviews(raw_text):
    try:
        text_content = raw_text.strip()
        entry = ast.literal_eval(text_content)
        
        rev_id = entry.get('reviewerID', 0)
        prod_asin = entry.get('asin', '0')
        rev_name = entry.get('reviewerName', 'unk')
        rating_score = entry.get('overall', 0)
        
        return (rev_id, prod_asin, rev_name, rating_score)
    except Exception:
        return None

reviews_source = sc.textFile("/home/beekfin/sna23mk048/BigData/Electronics_5.json") \
                   .map(extract_reviews) \
                   .filter(lambda x: x is not None)

reviews_source.take(5)

[('AO94DHGC771SJ', '0528881469', 'amazdnu', 5.0),
 ('AMO214LNFCEI4', '0528881469', 'Amazon Customer', 1.0),
 ('A3N7T0DY83Y4IG', '0528881469', 'C. A. Freeman', 3.0),
 ('A1H8PY3QHMQQA0', '0528881469', 'Dave M. Shaw "mack dave"', 2.0),
 ('A24EV6RXELQZ63', '0528881469', 'Wayne Smith', 1.0)]

In [ ]:
# расчет глобального среднего
all_ratings = reviews_source.map(lambda r: float(r[3])).filter(lambda v: v > 0)

print(f'Global Average Score: {all_ratings.mean()}')

Global Average Score: 4.222779228836578


## Задание 2
- Сопоставьте полученные данные из предыдущего пункта с наименованием товаров.

In [ ]:
# расчет среднего рейтинга для каждого продукта
asin_rating_pairs = reviews_source.map(lambda row: (row[1], (row[3], 1)))

aggregated_scores = asin_rating_pairs.reduceByKey(
    lambda acc, curr: (acc[0] + curr[0], acc[1] + curr[1])
)

avg_scores_by_asin = aggregated_scores.mapValues(lambda v: v[0] / v[1])

avg_scores_by_asin.take(5)

[('9985511476', 4.631578947368421),
 ('B00000J1E6', 2.142857142857143),
 ('B00000K13I', 4.533333333333333),
 ('B00001OPJV', 3.2),
 ('B00001P505', 4.434210526315789)]

In [ ]:
merged_data = metadata_rdd.rightOuterJoin(avg_scores_by_asin)

In [10]:
sample_merged = merged_data.take(5)

In [ ]:
for row in sample_merged:
    p_title = row[1][0]
    p_rating = row[1][1]
    print(f'Name: {p_title}, overall: {p_rating}')

Name: Targus PA400U DEFCON 1 Ultra Notebook Computer Security System, overall: 3.375
Name: Labtec LCS-1060 Spin 60 2-Piece Computer Speakers, overall: 4.666666666666667
Name: Sony DSC-S70 Cyber-shot 3.2MP Digital Camera with 3x Optical Zoom, overall: 4.777777777777778
Name: Uniden PRO520XL 40-Channel CB Radio, overall: 4.571428571428571
Name: Cisco-Linksys EFSP42 EtherFast 10/100 2-Port Switched PrintServer, overall: 3.3333333333333335


## Задание 3
- Сформируйте RDD товаров с рейтингом меньшим 3. Выведите топ-10 товаров с наименьшим рейтингом.

In [ ]:
worst_products = merged_data.filter(lambda x: x[1][1] <= 3) \
    .map(lambda x: (x[1][0], x[1][1])) \
    .sortBy(lambda x: x[1])

worst_products.take(5)

[('StarTech HDMISPL1HH 1 feet Standard HDMI Cable - 1x HDMI (M) to 2x HDMI (F) (Discontinued by Manufacturer)',
  1.0),
 ('ATI TV Wonder 200 PCI Video Card w/PVR Capabilities', 1.0),
 ('RCA DRC8335 DVD Recorder &amp; VCR Combo With Built-In Tuner', 1.0),
 ('GE 24746 Futura HDTV Ready Antenna', 1.0),
 ('NEEWER&reg; Photographic Barn Door &amp; Honeycomb Grid &amp; Gel Set for Alienbees Alienbee Flash',
  1.0)]

## Задание 4
- Сохраните результат в постоянное хранилище.

In [ ]:
worst_products.saveAsTextFile("output_results")